<a href="https://colab.research.google.com/github/pratyakshdixit19/Calendar-and-Remainder-App/blob/main/Language_Translation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Create a virtual environment (optional but recommended)
!python3 -m venv poetry_translation_env
!source poetry_translation_env/bin/activate  # On Windows use: poetry_translation_env\Scripts\activate
# Upgrade pip
!pip install --upgrade pip

# Install necessary libraries
!pip install transformers datasets torch sentencepiece


The virtual environment was not created successfully because ensurepip is not
available.  On Debian/Ubuntu systems, you need to install the python3-venv
package using the following command.

    apt install python3.10-venv

You may need to use sudo with that command.  After installing the python3-venv
package, recreate your virtual environment.

Failing command: /content/poetry_translation_env/bin/python3

/bin/bash: line 1: poetry_translation_env/bin/activate: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
import transformers
import datasets
import torch
import sentencepiece

print(transformers.__version__)
print(datasets.__version__)
print(torch.__version__)
print(sentencepiece.__version__)


4.44.2
3.0.2
2.5.0+cu121
0.2.0


In [4]:
import pandas as pd

# Load the dataset
data = pd.read_csv('hindi_english_poetry.csv')

# Display the first few rows
print(data.head())


                   hindi                                    english
0                    NaN                                        NaN
1     पवित्र जिसकी कामना                          Holy whose desire
2     उसका अतुल्य मोल है     His incredible marvellous valuation is
3  अर्पण हुए जो प्रेम से  contribution  occurred Who affection From
4   वह दान भी अनमोल है ।              This gift is also invaluable.


In [5]:
import re
import pandas as pd

def clean_text(text):
    # Check if the text is a string
    if isinstance(text, str):
        # Remove unwanted characters, if any
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = re.sub(r'\s+', ' ', text)     # Replace multiple spaces with single space
        text = text.strip()                  # Remove leading and trailing spaces
        return text
    else:
        # Handle non-string values (e.g., return an empty string or the original value)
        return ''

# Apply cleaning (convert the column to string type first)
data['hindi'] = data['hindi'].astype(str).apply(clean_text)
data['english'] = data['english'].astype(str).apply(clean_text)

print(data.head())


             hindi                                   english
0              nan                                       nan
1     पवतर जसक कमन                         Holy whose desire
2    उसक अतलय मल ह    His incredible marvellous valuation is
3  अरपण हए ज परम स  contribution occurred Who affection From
4   वह दन भ अनमल ह              This gift is also invaluable


In [6]:
from sklearn.model_selection import train_test_split

# First, split into train and temp (validation + test)
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)

# Then, split temp into validation and test
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_data)}")


Training samples: 440
Validation samples: 55
Test samples: 55


In [7]:
train_data.to_csv('train.csv', index=False)
val_data.to_csv('validation.csv', index=False)
test_data.to_csv('test.csv', index=False)


In [3]:
from transformers import MarianTokenizer, MarianMTModel

model_name = 'Helsinki-NLP/opus-mt-hi-en'

# Load tokenizer and model
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/304M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
from datasets import load_dataset

# Define the data files
data_files = {
    'train': 'train.csv',
    'validation': 'validation.csv',
    'test': 'test.csv'
}

# Load the dataset
dataset = load_dataset('csv', data_files=data_files)

# Inspect the dataset
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['hindi', 'english'],
        num_rows: 440
    })
    validation: Dataset({
        features: ['hindi', 'english'],
        num_rows: 55
    })
    test: Dataset({
        features: ['hindi', 'english'],
        num_rows: 55
    })
})


In [9]:
def tokenize_function(examples):
    # Ensure the input is a list of strings
    hindi_texts = [str(text) for text in examples['hindi']]

    # Tokenize the Hindi text
    inputs = tokenizer(hindi_texts, truncation=True, padding='max_length', max_length=128)

    # Tokenize the English text
    with tokenizer.as_target_tokenizer():
        english_texts = [str(text) for text in examples['english']]
        labels = tokenizer(english_texts, truncation=True, padding='max_length', max_length=128)

    inputs['labels'] = labels['input_ids']
    return inputs

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/440 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/55 [00:00<?, ? examples/s]

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

In [10]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 16
epochs = 5
learning_rate = 5e-5

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer

# Define data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator
)


data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator
)


In [12]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.441500,0.309874
2,0.263500,0.285572
3,0.232400,0.278379
4,0.197100,0.276799
5,0.192900,0.275781


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[61126]], 'forced_eos_token_id': 0}


TrainOutput(global_step=140, training_loss=0.6177894234657287, metrics={'train_runtime': 3405.0708, 'train_samples_per_second': 0.646, 'train_steps_per_second': 0.041, 'total_flos': 74576402841600.0, 'train_loss': 0.6177894234657287, 'epoch': 5.0})

In [13]:
model.save_pretrained('hindi-english-poetry-model')
tokenizer.save_pretrained('hindi-english-poetry-model')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[61126]], 'forced_eos_token_id': 0}


('hindi-english-poetry-model/tokenizer_config.json',
 'hindi-english-poetry-model/special_tokens_map.json',
 'hindi-english-poetry-model/vocab.json',
 'hindi-english-poetry-model/source.spm',
 'hindi-english-poetry-model/target.spm',
 'hindi-english-poetry-model/added_tokens.json')

In [14]:
from transformers import MarianMTModel, MarianTokenizer

# Load the fine-tuned model and tokenizer
tokenizer = MarianTokenizer.from_pretrained('hindi-english-poetry-model')
model = MarianMTModel.from_pretrained('hindi-english-poetry-model')


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [15]:
def translate(texts):
    # Tokenize the input texts
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=128)
    # Generate translations
    translated = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    # Decode the translations
    translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    return translations


In [16]:
!pip install --upgrade datasets
# Restart the kernel after upgrading the package.

from datasets import load_metric # updated import

# Load BLEU metric
bleu = load_metric('bleu')

def compute_bleu(predictions, references):
    # Tokenize and prepare data for BLEU
    references = [[ref.split()] for ref in references]
    predictions = [pred.split() for pred in predictions]
    return bleu.compute(predictions=predictions, references=references)

# Get test data
test_texts = test_data['hindi'].tolist()
test_references = test_data['english'].tolist()

# Generate translations
test_predictions = translate(test_texts)

# Compute BLEU score
bleu_score = compute_bleu(test_predictions, test_references)
print(f"BLEU score: {bleu_score['bleu']*100:.2f}")

ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.10/dist-packages/datasets/__init__.py)

In [17]:
# Already saved earlier using:
# model.save_pretrained('hindi-english-poetry-model')
# tokenizer.save_pretrained('hindi-english-poetry-model')

# To load for inference:
from transformers import MarianMTModel, MarianTokenizer

tokenizer = MarianTokenizer.from_pretrained('hindi-english-poetry-model')
model = MarianMTModel.from_pretrained('hindi-english-poetry-model')


In [18]:
def translate_poetry(hindi_text):
    inputs = tokenizer(hindi_text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    translated = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    english_translation = tokenizer.decode(translated[0], skip_special_tokens=True)
    return english_translation

# Example usage
hindi_poem = "प्राची मे अरुणिम की रेख देख पता हूँ"
english_translation = translate_poetry(hindi_poem)
print(f"English Translation: {english_translation}")

English Translation: I know I look forward to the care of the old man
